<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/Longformer_classifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primeiros testes com o Longformer

Duas formas de gerar um embedding com tamanho padronizado para representar toda a semântica de um texto: a camada de atenção global e a média dos tokens de saída.

In [1]:
import torch
from transformers import LongformerTokenizer, LongformerModel

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Liberar memória da GPU, se necessário
torch.cuda.empty_cache()

# Carregar o modelo e o tokenizador Longformer-large
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096")
model = LongformerModel.from_pretrained("allenai/longformer-large-4096").to(device)

# Habilitar gradient checkpointing para economizar memória
model.gradient_checkpointing_enable()

# Texto de exemplo
text = "Exemplo de texto longo que queremos classificar usando Longformer."

# Reduzir o comprimento máximo, se possível
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=2048)

# Definir a máscara de atenção: 1 para global attention no primeiro token (ou outro token especial)
attention_mask = torch.ones(inputs['input_ids'].shape, dtype=torch.long).to(device)  # Enviar para GPU
global_attention_mask = torch.zeros(inputs['input_ids'].shape, dtype=torch.long).to(device)  # Enviar para GPU
global_attention_mask[:, 0] = 1  # Dar atenção global ao primeiro token (posição 0)

# Enviar os inputs para a GPU
inputs = {key: value.to(device) for key, value in inputs.items()}

# Passar pelo modelo com a máscara de atenção global
outputs = model(input_ids=inputs['input_ids'], attention_mask=attention_mask, global_attention_mask=global_attention_mask)

# Extraímos o embedding do primeiro token (com atenção global)
global_attention_embedding = outputs.last_hidden_state[:, 0, :]  # Primeiro token com global attention

# Fazer a média dos embeddings de todos os tokens (global pooling)
output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)

# Verificar o formato dos embeddings
print('\n', global_attention_embedding.shape, '\n', output_mean_embedding.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



 torch.Size([1, 1024]) 
 torch.Size([1, 1024])


# Testando a capacidade de gerar resposta do Longformer

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from torch.cuda.amp import autocast

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Liberar memória da GPU
torch.cuda.empty_cache()

# Carregar um modelo e tokenizer compatíveis com o português
tokenizer = AutoTokenizer.from_pretrained('pierreguillou/bert-large-cased-squad-v1.1-portuguese')
model = AutoModelForQuestionAnswering.from_pretrained('pierreguillou/bert-large-cased-squad-v1.1-portuguese').to(device)

# Contexto detalhado em português
context = """
Longformer é um modelo de processamento de linguagem natural desenvolvido para lidar com grandes quantidades de texto.
Ele utiliza uma atenção local eficiente que permite processar até 4096 tokens em uma única passagem, o que é significativamente
maior do que outros modelos tradicionais. O Longformer é projetado para tarefas de leitura de documentos longos, perguntas e respostas,
sumarização, entre outras.
"""

# Pergunta em português
question = "Para que o Longformer foi projetado?"

# Tokenizar a pergunta e o contexto
inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Enviar os inputs para a GPU
inputs = {key: value.to(device) for key, value in inputs.items()}

# Obter a resposta do modelo usando o autocast atualizado
with torch.cuda.amp.autocast():
    outputs = model(**inputs)

start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Determinar os índices de início e fim da resposta
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

print(f"Start index: {start_index}, End index: {end_index}")

# Garantir que os índices estão corretos
if start_index <= end_index:
    # Decodificar a resposta
    answer = tokenizer.decode(inputs['input_ids'][0][start_index:end_index+1], skip_special_tokens=True, clean_up_tokenization_spaces=True)
else:
    answer = "Não foi possível encontrar uma resposta."

print(f"Resposta: {answer}")


Start index: 73, End index: 76
Resposta: leitura de documentos longos


<ipython-input-4-f98b3668b118>:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


# Aplicando no nosso problema de classificação

Carregando o arquivo de amostras e reduzindo a uma quantidade menor

In [1]:
import json
import random

# Carregar o arquivo JSON
with open('/content/realistic_denuncias_ambientais_5000_v2.json') as file:
    data = json.load(file)

# Selecionar 100 amostras aleatórias
sampled_data = random.sample(data, 200)

# Exibir as primeiras amostras selecionadas
sampled_data

[{'text': 'O gado está destruindo a nascente em Nordeste, pisoteando a área e acabando com a vegetação ao redor.',
  'tags': ['pisoteamento_nascente', 'APP', 'impacto ambiental', 'urgente']},
 {'text': 'A nascente da fazenda Fazenda Nova está sendo contaminada pelo gado, que está pisoteando a margem.',
  'tags': ['pisoteamento_nascente', 'APP', 'impacto ambiental', 'já ocorreu']},
 {'text': 'Queimaram tudo aqui na fazenda do Fazenda Esperança pra aumentar o espaço pro gado. Já aconteceu ano passado.',
  'tags': ['incendio_criminoso', 'APP', 'impacto ambiental', 'urgente']},
 {'text': 'Trabalhadores rurais estão aplicando agrotóxicos sem EPI na fazenda Sítio do João. Estão expostos a substâncias nocivas.',
  'tags': ['trabalhadores_sem_epi', 'APP', 'impacto ambiental', 'urgente']},
 {'text': 'Trabalhadores rurais estão aplicando agrotóxicos sem EPI na fazenda Sítio da Paz. Estão expostos a substâncias nocivas.',
  'tags': ['trabalhadores_sem_epi', 'APP', 'impacto ambiental', 'urgente']}

Gerando os embeddings para cada amostra

In [2]:
import torch
from transformers import LongformerTokenizer, LongformerModel

# Verificar se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Liberar memória da GPU, se necessário
torch.cuda.empty_cache()

# Carregar o modelo e o tokenizador Longformer-large
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096")
model = LongformerModel.from_pretrained("allenai/longformer-large-4096").to(device)

# Habilitar gradient checkpointing para economizar memória
model.gradient_checkpointing_enable()

def process_text(text):
    with torch.no_grad():  # Desabilitar o cálculo de gradientes
        # Reduzir o comprimento máximo, se possível
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=2048)

        # Definir a máscara de atenção: 1 para global attention no primeiro token (ou outro token especial)
        attention_mask = torch.ones(inputs['input_ids'].shape, dtype=torch.long).to(device)  # Enviar para GPU
        global_attention_mask = torch.zeros(inputs['input_ids'].shape, dtype=torch.long).to(device)  # Enviar para GPU
        global_attention_mask[:, 0] = 1  # Dar atenção global ao primeiro token (posição 0)

        # Enviar os inputs para a GPU
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Passar pelo modelo com a máscara de atenção global
        outputs = model(input_ids=inputs['input_ids'], attention_mask=attention_mask, global_attention_mask=global_attention_mask)

        # Extraímos o embedding do primeiro token (com atenção global)
        global_attention_embedding = outputs.last_hidden_state[:, 0, :]  # Primeiro token com global attention

        # Fazer a média dos embeddings de todos os tokens (global pooling)
        output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)

        # Mover os embeddings para a CPU
        global_attention_embedding = global_attention_embedding.cpu()
        output_mean_embedding = output_mean_embedding.cpu()

        # Sincronizar CUDA para garantir que a GPU terminou o processamento antes de prosseguir
        torch.cuda.synchronize()

        # Limpar variáveis não utilizadas e liberar memória da GPU
        del inputs, attention_mask, global_attention_mask, outputs
        torch.cuda.empty_cache()

        return global_attention_embedding, output_mean_embedding

# Listas para armazenar embeddings
ga_embeddings = []
om_embeddings = []

# Processar as amostras
for item in sampled_data:
    ga_embedding, om_embedding = process_text(item['text'])

    # Adicionar embeddings às listas
    ga_embeddings.append(ga_embedding)
    om_embeddings.append(om_embedding)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Verificando os textos próximos utilizando a similiaridade de cosseno

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate_similarity(embeddings, query_index=0, top_n=20):
    # Converter listas de embeddings para numpy arrays 2D
    embeddings = np.array([embedding.squeeze().numpy() for embedding in embeddings])  # Converter para 2D

    # Pegar o embedding do texto de interesse (query_index)
    query_embedding = embeddings[query_index]

    # Calcular a similaridade de cosseno entre o texto de interesse e todos os outros
    similarities = cosine_similarity([query_embedding], embeddings)

    # Classificar os textos por similaridade (em ordem decrescente)
    ranked_indices = np.argsort(similarities[0])[::-1]

    # Retornar os top_n textos mais semelhantes
    return ranked_indices[:top_n], similarities[0][ranked_indices[:top_n]]

print("GLOBAL ATTENTION EMBEDDING\n")

top_indices, top_similarities = evaluate_similarity(ga_embeddings)

for i in range(len(top_indices)):
    print(f"Texto {top_indices[i]}: Similaridade = {top_similarities[i]}\n{sampled_data[top_indices[i]]['text']}\n\n")

print("\nOUTPUT MEAN EMBEDDING\n")

top_indices, top_similarities = evaluate_similarity(om_embeddings)

for i in range(len(top_indices)):
    print(f"Texto {top_indices[i]}: Similaridade = {top_similarities[i]}\n{sampled_data[top_indices[i]]['text']}\n\n")

GLOBAL ATTENTION EMBEDDING

Texto 0: Similaridade = 0.9999998211860657
O gado está destruindo a nascente em Nordeste, pisoteando a área e acabando com a vegetação ao redor.


Texto 102: Similaridade = 0.9999998211860657
O gado está destruindo a nascente em Nordeste, pisoteando a área e acabando com a vegetação ao redor.


Texto 26: Similaridade = 0.9999998211860657
O gado está destruindo a nascente em Nordeste, pisoteando a área e acabando com a vegetação ao redor.


Texto 40: Similaridade = 0.999997079372406
O gado está destruindo a nascente em Sudeste, pisoteando a área e acabando com a vegetação ao redor.


Texto 153: Similaridade = 0.999997079372406
O gado está destruindo a nascente em Sudeste, pisoteando a área e acabando com a vegetação ao redor.


Texto 114: Similaridade = 0.9999921321868896
O gado está destruindo a nascente em Norte, pisoteando a área e acabando com a vegetação ao redor.


Texto 106: Similaridade = 0.9999880194664001
O gado está destruindo a nascente em Sul de 